# Exploratory Data Analysis
spiegare quali siano gli obbiettivi per i quali stiamo preparando i dataset



## Import Packages
* Il pacchetto **numpy** serve a lavorare con valori e array numerici. Performa efficienti conti matematici e statistici.
* Il pacchetto **pandas** serve a manipolare, pulire e gestire dati strutturati attraverso strutture come i DataFrame.
* Il pacchetto **geopandas** è un'estensione del pacchetto *pandas* e serve per gestire dati geospaziali e visualizzarli in mappe.

In [65]:
import pandas as pd
import numpy as np
import geopandas as gp

## Importa Data

Il primo DataFrame contiene i dati sui consumi delle linee elettriche di tutto il Trentino.  
Per ogni linea elettrica viene fornito il dato di consumo ogni 10 minuti per l'intero mese.  
I dati sono per il mese di Novembre 2013 (il processo verrà ripetuto per il mese di Dicembre 2013).


Il secondo DataFrame contiene i dati riguardanti le ubicazioni e linee elettriche presenti in ogni zona del Trentino.  
Per ogni zona del Trentino (identificata da *SQUAREID*) vengono associate le linee elettriche da lì passanti (*LINESET*) e per ogni combinazione di zona e linea elettrica viene associato il numero di ubicazioni presenti (*NR_UBI*)



In [81]:
set_df = pd.read_csv("./data/processed/SET-nov-2013.csv")
lin_df = pd.read_csv("./data/processed/line.csv")
set_df[:5]

,LINESET,DATE,POWER
0,DG1000420,01/11/2013 00:00,37.439999
1,DG1000420,01/11/2013 00:10,37.439999
2,DG1000420,01/11/2013 00:20,36.000000
3,DG1000420,01/11/2013 00:30,35.279999
4,DG1000420,01/11/2013 00:40,35.279999


## Trasformazione colonna *DATE*
Trasformiamo la colonna *DATE* in tre colonne separate che indicano il giorno (*DAY*), i minuti (*MIN*) e il giorno della settimana (*WEEK_DAY*).

In [82]:
set_df["DATE"] = pd.to_datetime(set_df["DATE"], format="%d/%m/%Y %H:%M", dayfirst=True)
set_df["DAY"] = pd.to_datetime(set_df["DATE"]).dt.day
set_df["MINUTE"] = pd.to_datetime(set_df["DATE"]).dt.hour * 60 + pd.to_datetime(set_df["DATE"]).dt.minute
set_df["WEEK_DAY"] = pd.to_datetime(set_df["DATE"]).dt.day_of_week
del set_df["DATE"]
set_df[:5]

,LINESET,POWER,DAY,MINUTE,WEEK_DAY
0,DG1000420,37.439999,1,0,4
1,DG1000420,37.439999,1,10,4
2,DG1000420,36.000000,1,20,4
3,DG1000420,35.279999,1,30,4
4,DG1000420,35.279999,1,40,4


## AA
Quello che vogliamo vedere come prima 

??Ora voglio associare i dati dei due DataFrame organizzarli in base alla zona??

Ora divido il data frame *lin_df* in modo da poter contare il numero di ubicazioni presenti per ogni linea elettrica *LINESET* e contare il numero di ubicazioni presenti per ogni casella *SQUAREID*.

In [77]:
lin_df[:4]

,SQUAREID,LINESET,NR_UBI
0,155,DG1031221,4
1,272,DG1031221,72
2,273,DG1031221,21
3,276,DG1003117,34


Per prima cosa creo un DataFrame temporaneo dove ad ogni linea *LINESET* associo il numero di ubicazioni passanti per essa:

In [69]:
temp_df_line = lin_df.groupby("LINESET")["NR_UBI"].sum().reset_index()
temp_df_line = temp_df_line.rename(columns={"NR_UBI": "UBI_X_LINE"})
temp_df_line[:5]

,LINESET,UBI_X_LINE
0,DG1000011,6
1,DG1000023,62
2,DG1000420,2133
3,DG1000421,1424
4,DG1000422,3423


Poi creo un DataFrame temporaneo dove ad ogni casella *SQUAREID* associo il numero di ubicazioni presenti:

In [70]:
temp_df_square = lin_df.groupby("SQUAREID")["NR_UBI"].sum().reset_index()
temp_df_square = temp_df_square.rename(columns={"NR_UBI": "UBI_X_SQUARE"})
temp_df_square[:5]

,SQUAREID,UBI_X_SQUARE
0,155,4
1,272,72
2,273,21
3,276,34
4,277,3


Dopo aver separato *lin_df* nei due data frame temporanei, li riunisco ad data frame *lin_df* e a sua volta li unisco in un data frame finale insieme a *set_df* organizzandoli sotto *LINESET*.

In [71]:
final_df1=pd.merge(lin_df,temp_df_line,on="LINESET")
final_df1[:5]

,SQUAREID,LINESET,NR_UBI,UBI_X_LINE
0,155,DG1031221,4,1380
1,272,DG1031221,72,1380
2,273,DG1031221,21,1380
3,389,DG1031221,65,1380
4,390,DG1031221,8,1380


In [72]:

final_df2=pd.merge(final_df1,temp_df_square,on="SQUAREID")
final_df2[:5]

,SQUAREID,LINESET,NR_UBI,UBI_X_LINE,UBI_X_SQUARE
0,155,DG1031221,4,1380,4
1,272,DG1031221,72,1380,72
2,273,DG1031221,21,1380,21
3,389,DG1031221,65,1380,65
4,390,DG1031221,8,1380,8


In [83]:
final_df = pd.merge(final_df2,set_df, on = "LINESET")
final_df#[:5]

,SQUAREID,LINESET,NR_UBI,UBI_X_LINE,UBI_X_SQUARE,POWER,DAY,MINUTE,WEEK_DAY
0,155,DG1031221,4,1380,4,35.040001,1,0,4
1,155,DG1031221,4,1380,4,30.000000,1,10,4
2,155,DG1031221,4,1380,4,32.399998,1,20,4
3,155,DG1031221,4,1380,4,30.840000,1,30,4
4,155,DG1031221,4,1380,4,30.599998,1,40,4
...,...,...,...,...,...,...,...,...,...
10928875,10984,DG1013504,2,74,2,160.225006,30,1390,5
10928876,10984,DG1013504,2,74,2,161.699997,30,1400,5
10928877,10984,DG1013504,2,74,2,159.600006,30,1410,5
10928878,10984,DG1013504,2,74,2,161.600006,30,1420,5
